In [ ]:
import pkg_resources
import pickle
import nltk
import re, string, json
from tqdm.notebook import tqdm
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

In [ ]:
from keras import models
from keras import layers
from keras import regularizers
from keras.preprocessing.text import Tokenizer

In [ ]:
import os
from google.colab import drive

drive.mount('/content/gdrive')
print(os.getcwd())

Mounted at /content/gdrive
/content


In [ ]:
os.chdir("/content/gdrive/My Drive/Colab Notebooks/Dataset")
os.getcwd() 

'/content/gdrive/My Drive/Colab Notebooks/Dataset'

In [ ]:
# Creating new dataset

In [ ]:
with open("MovieReviews.txt", "r", encoding = "utf-8") as f:
    data = f.read()
    
data[:1000]

'I admit, the great majority of films released before say 1933 are just not for me. Of the dozen or so "major" silents I have viewed, one I loved (The Crowd), and two were very good (The Last Command and City Lights, that latter Chaplin circa 1931).<br /><br />So I was apprehensive about this one, and humor is often difficult to appreciate (uh, enjoy) decades later. I did like the lead actors, but thought little of the film.<br /><br />One intriguing sequence. Early on, the guys are supposed to get "de-loused" and for about three minutes, fully dressed, do some schtick. In the background, perhaps three dozen men pass by, all naked, white and black (WWI ?), and for most, their butts, part or full backside, are shown. Was this an early variation of beefcake courtesy of Howard Hughes?This is a very strange film that was long thought to be forgotten. It\'s the story of two American Army buddies, William Boyd (aka "Hopalong Cassidy") and Louis Wolheim, and their adventures as they manage to

In [ ]:
def split_to_sentences(data, point):
    """
    Split data by linebreak point(can be '\n')
    
    Args:
        data: str
    
    Returns:
        A list of sentences
    """
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    sentences = data.split(point)
    ### END CODE HERE ###
    
    # Additional clearning (This part is already implemented)
    # - Remove leading and trailing spaces from each sentence
    # - Drop sentences if they are empty strings.
    sentences = [s.strip() for s in sentences]
    sentences = [s for s in sentences if len(s) > 3]
    
    return sentences 

In [ ]:
def normalize_contractions(sentence_list):
    contraction_list = json.loads(open('english_contractions.json', 'r').read())
    norm_sents = []
    print("Normalizing contractions")
    for sentence in tqdm(sentence_list):
        norm_sents.append(_normalize_contractions_text(sentence, contraction_list))
    # print(norm_sents)
    return norm_sents

def _normalize_contractions_text(text, contractions):
    """
    This function normalizes english contractions.
    """
    new_token_list = []
    token_list = text.split()
    for word_pos in range(len(token_list)):
        word = token_list[word_pos]
        first_upper = False
        if word[0].isupper():
            first_upper = True
        if word.lower() in contractions:
            replacement = contractions[word.lower()]
            if first_upper:
                replacement = replacement[0].upper()+replacement[1:]
            replacement_tokens = replacement.split()
            if len(replacement_tokens)>1:
                new_token_list.append(replacement_tokens[0])
                new_token_list.append(replacement_tokens[1])
            else:
                new_token_list.append(replacement_tokens[0])
        else:
            new_token_list.append(word)
    sentence = " ".join(new_token_list).strip(" ")
    return sentence

In [ ]:
def tokenize_twitter(sentences):
    """
    Tokenize sentences into tokens (words)
    
    Args:
        sentences: List of strings
    
    Returns:
        List of lists of tokens
    """
    print("Starting Cleaning Process")
    tokenized_sentences = []
    for sentence in tqdm(sentences):
        
        # Convert to lowercase letters
        sentence = sentence.lower()
        sentence = cleanhtml(sentence)
        sentence = _replace_urls(sentence)
        sentence = re.sub(r'[^a-zA-Z]', ' ', sentence)
        # sentence = re.sub(r'[^\w\s]',' ',sentence)
        sentence = re.sub(' +', ' ', sentence)
        # print(sentence)
        tokenized = nltk.word_tokenize(sentence)
        
        # append the list of words to the list of lists
        tokenized_sentences.append(tokenized)
    
    ### END CODE HERE ###
    
    return tokenized_sentences


def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def _replace_urls(text):
    text = re.sub(r"\S*https?:\S*", "", text)
    return text


In [ ]:
def lemma(sentences):
    lemmatizer = WordNetLemmatizer()
    print("Starting Lemmatization and Tokenizing Process")
    final = []
    for sentence in tqdm(sentences):
        lemmatized_output = [lemmatizer.lemmatize(w) for w in sentence]
        # print(lemmatized_output)
        word_list = " ".join(lemmatized_output).strip(" ")
        word_lists = word_list + "."
        # print(word_lists)
        final.append(word_lists)
    
    return final

In [ ]:
lemmatizer = WordNetLemmatizer()
print(lemmatizer.lemmatize("was"))

def build(sentences):
  final = []
  print("Joining sentences")
  for sentence in tqdm(sentences):
    word_list = " ".join(sentence).strip(" ")
    word_lists = word_list + "."
    final.append(word_lists)

  return final

wa


In [ ]:
def normalization_pipeline(data):
    print("Starting Normalization Process")
    sentences = split_to_sentences(data, ".")
    sentences = normalize_contractions(sentences)
    sentences = tokenize_twitter(sentences)
    # sentences = lemma(sentences)
    sentences = build(sentences)
    print("Normalization Process Finished")
    return sentences

In [ ]:
# nltk.download('punkt')
# nltk.download('wordnet')

In [ ]:
prefinal = normalization_pipeline(data)

Starting Normalization Process
Normalizing contractions



Starting Cleaning Process



Joining sentences



Normalization Process Finished


In [ ]:
prefinal[:9]

['i admit the great majority of films released before say are just not for me.',
 'of the dozen or so major silents i have viewed one i loved the crowd and two we are very good the last command and city lights that latter chaplin circa.',
 'so i was apprehensive about this one and humor is often difficult to appreciate uh enjoy decades later.',
 'i did like the lead actors but thought little of the film.',
 'one intriguing sequence.',
 'early on the guys are supposed to get de loused and for about three minutes fully dressed do some schtick.',
 'in the background perhaps three dozen men pass by all naked white and black wwi and for most their butts part or full backside are shown.',
 'was this an early variation of beefcake courtesy of howard hughes this is a very strange film that was long thought to be forgotten.',
 'it is the story of two american army buddies william boyd aka hopalong cassidy and louis wolheim and their adventures as they manage to escape from a german prison camp 

In [ ]:
# building new data
new_data = ' '.join([t for t in prefinal if t])

In [ ]:
len(new_data.split())

16264927

In [ ]:
file1 = open("Pre Data/Movies.txt","a")
file1.write(new_data)
file1.close()